## Energy consumption forecasting model

 The data ranges from the beginning of 2023 to July 2024, and the forecasted day (day) is from 2024. To monitor data leakage, meaning that for the forecasted day, no data from the forecasted day or the preceding day should be used during training or forecasting.

All data is hourly, where time_local indicates the start of the corresponding hour in local time. The data is real.

Tables downloaded from https://dashboard.elering.ee/et/system/with-plan/production-consumption and ilm.ee


In [34]:
from datetime import datetime, timedelta 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [ ]:
weather_data = pd.read_excel("weather_data.xlsx", parse_dates=['time_local'])
print(weather_data.isnull().sum().sum())

# Since the number of rows with missing data is very small compared to the total number of rows,
# I decided to just remove them and not take them into account in the model to avoid potential errors in the data. 
# Before this, I checked in Excel to ensure there were no excessively large or impossibly small values.
# The weather data table includes separate location data, which I have grouped by datetime. 
# This grouping was done because I also have energy consumption data for the entire country of Estonia.


weather_data_cleared = weather_data.drop(columns=['location_name']).dropna().groupby('time_local').mean().reset_index()
weather_data_cleared.head()

36


,time_local,air_temperature,visibility,dew_point,feels_like_temp,pressure,relative_humidity,wind_speed,wind_from_direction
0,2023-01-01 01:00:00,4.445556,6660.555556,3.721667,0.780556,998.333333,95.111111,4.975000,221.888889
1,2023-01-01 02:00:00,4.530556,8444.944444,3.730000,0.912222,998.277778,94.611111,4.891111,227.166667
2,2023-01-01 03:00:00,4.396111,8564.277778,3.641667,0.788889,998.388889,94.888889,4.775000,233.055556
3,2023-01-01 04:00:00,4.514444,9160.333333,3.572222,0.992222,998.777778,93.666667,4.706111,242.944444
4,2023-01-01 05:00:00,4.276111,8964.111111,3.252222,0.961667,998.722222,93.111111,4.158889,251.444444


In [3]:
consumption_data = pd.read_excel("consumption_data.xlsx")
print(consumption_data.isnull().sum().sum())
consumption_data.dropna(inplace=True)
consumption_data.head()

3200


,Ajatempel (UTC),Kuupäev (Eesti aeg),Tarbimine
0,1.672524e+09,2023-01-01 00:00:00,798.2
1,1.672528e+09,2023-01-01 01:00:00,793.4
2,1.672531e+09,2023-01-01 02:00:00,776.5
3,1.672535e+09,2023-01-01 03:00:00,757.0
4,1.672538e+09,2023-01-01 04:00:00,743.7


In [ ]:
def forecast_system_consumption(day: datetime.date, weather_df: pd.DataFrame, consumption_df: pd.DataFrame) -> list:
    day_datetime = pd.Timestamp(day)

    # Avoid data leakage by only using past consumption data (excluding 'day' and 'day-1')
    consumption_cutoff = day_datetime - timedelta(days=1)
    consumption_df_for_forecast = consumption_df[consumption_df["Kuupäev (Eesti aeg)"] < consumption_cutoff]

    # Weather data can be used entirely (no data leakage concerns)
    weather_df_for_forecast = weather_df
    
    # Merge past consumption with weather data
    df = pd.merge(consumption_df_for_forecast, weather_df_for_forecast, left_on="Kuupäev (Eesti aeg)", right_on="time_local", how="inner")
    
    # Feature Engineering
    df['day_of_week'] = df['Kuupäev (Eesti aeg)'].dt.dayofweek
    df['month'] = df['Kuupäev (Eesti aeg)'].dt.month
    df['day_of_year'] = df['Kuupäev (Eesti aeg)'].dt.dayofyear
    df['hour'] = df['Kuupäev (Eesti aeg)'].dt.hour

    # Define Features and Target
    features = ['air_temperature', 'relative_humidity', 'wind_speed', 'pressure', 'visibility', 
                'feels_like_temp', 'dew_point', 'wind_from_direction', 'day_of_week', 'month', 
                'day_of_year', 'hour']
    target = 'Tarbimine'


    X = df[features]
    y = df[target]

    # Split data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)

    # Standardize features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    # Model Selection - Random Forest, I have chosen Random Forest Regression cause it has the best MAE
    model = RandomForestRegressor(n_estimators=250, random_state=1)
    model.fit(X_train_scaled, y_train)

    # Model Evaluation
    val_predictions = model.predict(X_val_scaled)
    val_mae = mean_absolute_error(y_val, val_predictions)
    print(f"Validation MAE: {val_mae:,.2f}")

    # Generate future timestamps for the day to be predicted
    future_hours = pd.date_range(start=day_datetime, periods=24, freq='h') 
    future_df = pd.DataFrame({'datetime': future_hours})
    
    # Extract features for the future predictions
    future_df['day_of_week'] = future_df['datetime'].dt.dayofweek
    future_df['month'] = future_df['datetime'].dt.month
    future_df['day_of_year'] = future_df['datetime'].dt.dayofyear
    future_df['hour'] = future_df['datetime'].dt.hour

    future_df = pd.merge(future_df, weather_df_for_forecast, left_on='datetime', right_on='time_local', how='left')

    X_future = future_df[features]
    X_future_scaled = scaler.transform(X_future)

    # Predict future consumption
    model_output = model.predict(X_future_scaled)
    
    return model_output

In [ ]:
def model_metrics(day: datetime.date, consumption_df: pd.DataFrame, model_output: list) -> pd.DataFrame:
    # MAPE and MAE checking

    actual_consumption = consumption_df.loc[consumption_df["Kuupäev (Eesti aeg)"].dt.date == day, "Tarbimine"].to_list()

    comparison_df = pd.DataFrame({"true": actual_consumption, "pred": model_output})

    comparison_df["MAE"] = np.abs(comparison_df.true - comparison_df.pred)
    comparison_df["MAPE"] = np.abs(comparison_df.MAE / comparison_df.true)

    metrics = comparison_df.mean()

    return metrics


In [ ]:
model_output = forecast_system_consumption(datetime(2023, 6, 2).date(), weather_data_cleared, consumption_data)
print(model_output)

model_metrics(datetime(2023, 6, 2).date(), consumption_data, model_output)

Validation MAE: 22.83
[ 686.1604  673.3596  671.9716  669.804   666.544   669.7108  789.5032
  941.5864  953.4804  961.7604  985.6988  994.4612 1003.186  1011.5412
 1003.8896  991.7408  960.1304  925.9128  925.914   926.3764  908.1748
  829.996   807.7808  760.9948]


true    891.904167
pred    863.319933
MAE      45.520300
MAPE      0.046808
dtype: float64